In [1]:
# If Colab
!git clone https://github.com/goncalogiga/LOTClass
!pip install LOTClass/.
!pip install transformers
!pip install sentencepiece

fatal: destination path 'LOTClass' already exists and is not an empty directory.
ERROR: Directory 'LOTClass/.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [2]:
# If Colab -> load the mails as zips
!unzip cleaned_mails.zip
!unzip cleaned_mails_labeled.zip

unzip:  cannot find or open cleaned_mails.zip, cleaned_mails.zip.zip or cleaned_mails.zip.ZIP.
unzip:  cannot find or open cleaned_mails_labeled.zip, cleaned_mails_labeled.zip.zip or cleaned_mails_labeled.zip.ZIP.


In [3]:
import pandas as pd

# Loading the dataset
train_set = pd.read_csv("data/cleaned_mails.csv")
test_set = pd.read_csv("data/cleaned_mails_labeled.csv")
test_set = test_set[(test_set["Catégorie"] != "Réclamation") & (test_set["Catégorie"] != "Déménagement") & (test_set["Catégorie"] != "Contrat – Coordonnées personnelles")]

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/goncalo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from LOTClass.french.LOTClassModel import LOTClassifier
from LOTClass.french.config import LOTClassConfig

labels = [
          ['releve compteur'], 
          ['espace client'], 
          ['facture factures montant mensualisation virement virements bancaire bancaires']
  ]

args = LOTClassConfig(overwrite_dataset=True, train_batch_size=4, gpus=1, 
                      eval_batch_size=4, accum_steps=32,
                      test_file="test_file.txt", 
                      test_label_file="test_label_file.txt")

model = LOTClassifier(path="datasets/mails/", labels=labels, 
                      args=args)

X_test = test_set["Corps"]
y_test = test_set["Catégorie"]

model.build_test_dataset(list(X_test), list(y_test))

In [6]:
train_set

,Corps
0,n'arrive joindre car mes horaires travail sont...
1,comme voici retour courriel justificatif maria...
2,chere cher serions reconnaissantes vouloir pro...
3,pouvez passe cela fait fois j'essaie connecter...
4,espace suis actuellement titulaire abonnements...
...,...
574,voici semaines m'a ete receptionne petite mais...
575,électronique voila clair maintenant effet sach...
576,voudrais informer vais quitter appartement sou...
577,recu facture annuelle souhaiterai comprendre p...


In [7]:
test_set

,Corps,Catégorie
1,comme voici retour courriel justificatif maria...,Espace client
3,espace suis actuellement titulaire abonnements...,Espace client
4,facture recu document cependant pensais factur...,Relève de compteur
7,bonsoir virement facture ref ete virement banc...,Facture – Paiement
9,voici l'index releve jour bonne,Relève de compteur
...,...,...
279,mes factures soient car certaines sommes sont ...,Facture – Paiement
280,compte internet avoir reussi creer compte pers...,Espace client
281,numéro ecris concernant nos locataire locatair...,Relève de compteur
282,viens d'emmenager souscrire internet depuis se...,Espace client


In [8]:
model.fit(list(train_set["Corps"]))

Effective training batch size: 128


Downloading: 100%|██████████| 792k/792k [00:00<00:00, 1.48MB/s]


In [ ]:
y_true, y_pred = model.classification_report(list(y_test))

NameError: name 'model' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
sns.heatmap(confusion_matrix(y_true, y_pred), annot=True)